# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

browser = webdriver.Edge()
browser.get('https://google.com.br')

# get dollar price
googleSearchInput = browser.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
googleSearchInput.send_keys('cotação dolar')
googleSearchInput.send_keys(Keys.ENTER)

dolarValue = browser.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(dolarValue)

# get euro price
browser.get('https://google.com.br')
googleSearchInput = browser.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
googleSearchInput.send_keys('cotação euro')
googleSearchInput.send_keys(Keys.ENTER)

euroValue = browser.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(euroValue)

# get gold price
browser.get('https://www.melhorcambio.com/ouro-hoje')
goldValue = browser.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
goldValue = goldValue.replace(',', '.')
print(goldValue)

browser.quit()




5.1971
5.1529
288.46


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [9]:
import pandas as pd

tab = pd.read_excel('Produtos.xlsx')
display(tab)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [11]:
dollarRows = tab['Moeda'] == 'Dólar'
euroRows = tab['Moeda'] == 'Euro'
goldRows = tab['Moeda'] == 'Ouro'

tab.loc[dollarRows, 'Cotação'] = float(dolarValue)
tab.loc[euroRows, 'Cotação'] = float(euroValue)
tab.loc[goldRows, 'Cotação'] = float(goldValue)

tab['Preço de Compra'] = tab['Preço Original'] * tab['Cotação']
tab['Preço de Venda'] = tab['Preço de Compra'] * tab['Margem']

display(tab)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.1971,5197.048029,1.40,7275.867241
1,Carro Renault,4500.00,Euro,5.1529,23188.050000,2.00,46376.100000
2,Notebook Dell,899.99,Dólar,5.1971,4677.338029,1.70,7951.474649
3,IPhone,799.00,Dólar,5.1971,4152.482900,1.70,7059.220930
4,Carro Fiat,3000.00,Euro,5.1529,15458.700000,1.90,29371.530000
5,Celular Xiaomi,480.48,Dólar,5.1971,2497.102608,2.00,4994.205216
6,Joia 20g,20.00,Ouro,288.4600,5769.200000,1.15,6634.580000


### Agora vamos exportar a nova base de preços atualizada

In [12]:
tab.to_excel('Produtos Novo.xlsx', index=False)